In [1]:
# multi-class classification with Keras
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

2021-08-30 05:19:49.164430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-30 05:19:49.164508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
raw_df = pandas.read_csv('../../../test-builds/llvm/datasets/model-aug-29.csv')
dataset = dataframe.values

In [15]:
cleaned_df = raw_df.copy()

# don't want the 'fullpath', 'modulepath', 'function' columns
cleaned_df = cleaned_df.drop(['fullpath', 'modulepath', 'function'], axis=1)

def clean_labels(x):
    if x.strip() == 'O1':
        return 1 
    elif x.strip() == 'O2':
        return 2 
    elif x.strip() == 'O3':
        return 3
    else:
        return 0
    

cleaned_df['label'] = cleaned_df['label'].map(clean_labels)

cleaned_df.head()    

,BasicBlockCount,BasicBlockWithSingleSuccessor,BasicBlockWithTwoSuccessors,BasicBlockWithMoreThanTwoSuccessors,BasicBlockWithSinglePredecessor,BasicBlockWithTwoPredecessors,BasicBlockWithMoreThanTwoPredecessors,BigBasicBlock,MediumBasicBlock,SmallBasicBlock,...,CastInstCount,FloatingPointInstCount,IntegerInstCount,LoadInstCount,StoreInstCount,CallInstCount,PHIInstCount,AllocaInstCount,GEPInstCount,label
0,100,55,33,11,55,33,11,0,22,78,...,110,0,0,110,176,222,0,44,143,2
1,7,3,3,0,4,1,1,0,1,6,...,5,0,1,4,8,7,1,1,12,0
2,3,1,1,0,1,1,0,0,1,2,...,5,0,0,4,7,9,0,2,11,0
3,7,3,3,0,4,1,1,0,1,6,...,5,0,1,5,8,7,1,1,11,0
4,148,65,74,0,95,38,14,0,7,141,...,86,0,48,94,53,147,4,33,92,2


In [16]:
Y = cleaned_df.pop('label').values
X = cleaned_df.values

In [17]:
# convert integers to dummy variables (i.e. one hot encoded)
encoded_Y = np_utils.to_categorical(Y)

In [23]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=27, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))